In [63]:
import pandas
import os
from py_linq import Enumerable
os.environ['R_HOME'] = 'C:\Program Files\R\R-4.3.1'

from rpy2.robjects.packages import importr

base = importr('base')
lsafun = importr('LSAfun')
lsa = importr('lsa')

In [69]:
input_data = pandas.read_csv("Totally Real Conversation Data - Sheet1.csv")
aggregated_text_by_participant = Enumerable([row for _, row in input_data.iterrows()]) \
    .group_by(key_names=['participant_id', 'f_id', 'm_id', 'date.number'], key=lambda row: (row['ID'], row['F.ID'], row['M.ID'], row['Date.Number'])) \
    .select(lambda group: {'ID': group.key, 'Text': group \
        .select(lambda row: row['Event']) \
        .aggregate(lambda a, b: f"{a}\n{b}")})
    
# LSA requires text *files* instead of just text for god knows why
for entry in aggregated_text_by_participant:
    print(entry)
    participant_text_file = open(f"participant_text_for_{entry['ID'].participant_id}.txt", 'w')
    participant_text_file.write(entry['Text'])
    participant_text_file.close()

{'ID': {'participant_id': 'Juliet', 'f_id': 'Juliet', 'm_id': 'Romeo', 'date.number': 1}, 'Text': "What man art thou that, thus bescreened in night,\nSo stumblest on my counsel?\nMy ears have not yet drunk a hundred words\nOf that tongue's uttering, yet I know the sound.\nArt thou not Romeo and a Montague?\nHow camest thou hither, tell me, and wherefore?\nThe orchard walls are high and hard to climb,\nAnd the place death, considering who thou art,\nIf any of my kinsmen find thee here."}
{'ID': {'participant_id': 'Romeo', 'f_id': 'Juliet', 'm_id': 'Romeo', 'date.number': 1}, 'Text': "I take thee at thy word.\nCall me but love, and I'll be new baptized;\nHenceforth I never will be Romeo.\nBy a name\nI know not how to tell thee who I am.\nMy name, dear saint, is hateful to myself,\nBecause it is an enemy to thee.\nHad I it written, I would tear the word.\nNeither, fair maid, if either thee dislike."}


In [70]:
tvectors = lsa.textmatrix(base.getwd())
result = lsafun.costring(*aggregated_text_by_participant.select(lambda group: group['Text']), tvectors=tvectors)
print(result)

Note: not all elements in x were found in rownames(tvectors)

Note: not all elements in y were found in rownames(tvectors)

[0.87993471]
